In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("lrex").getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
# Creating instance for linear regression model
lr = LinearRegression (featuresCol = 'features',labelCol='label', predictionCol='prediction')

In [7]:
# Training the model
lrModel = lr.fit(training)

In [8]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lrModel.intercept

0.14228558260358093

In [10]:
# Finding some values with the use of summary of the model
training_summary = lrModel.summary

In [11]:
training_summary.r2

0.027839179518600154

In [12]:
training_summary.rootMeanSquaredError

10.16309157133015

In [14]:
# Reading whole data set into the variable 
all_data = spark.read.format ('libsvm').load('sample_linear_regression_data.txt')
# Training and test data split using randomSplit function
split_object = all_data.randomSplit([0.7,0.3])
split_object

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [16]:
# Instead of storing in split object we can store the train data and test data seperately
train_data , test_data = all_data.randomSplit([0.7,0.3])

In [19]:
train_data.show()
train_data.describe().show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|
|-16.151349351277112|(10,[0,1,2,3,4,5,...|
| -16.08565904102149|(10,[0,1,2,3,4,5,...|
+----------

In [20]:
# Creating actual correct model
correct_model = lr.fit (train_data)

In [21]:
# Evaluating how the train data fits against the test data
test_results = correct_model.evaluate (test_data)

In [23]:
# Using inbuilt functions with the evaluated train test data
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -29.38672339055108|
| -22.21863026166891|
|-23.272616705953006|
|-21.707687169171006|
|-20.401138148973565|
| -18.67736661368331|
|-19.788286908590038|
| -15.26948950040167|
| -20.33174119197154|
|-18.246160439454723|
|  -17.4968783407356|
| -16.72296543974181|
|  -12.5656083400093|
|-11.349033368473453|
|-11.164520482260682|
| -12.25561997132449|
|-14.640124802125214|
| -9.111047084014123|
|-13.733166094665917|
|-13.819511945104468|
+-------------------+
only showing top 20 rows



In [25]:
# Creating the unlabelled data for the prediction using prediction function
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [27]:
# Using transform function instead of evaluating function
predictions = correct_model.transform(unlabeled_data)
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  1.3407053527744464|
|(10,[0,1,2,3,4,5,...|  -1.292253828654062|
|(10,[0,1,2,3,4,5,...|-0.21482341498350732|
|(10,[0,1,2,3,4,5,...| 0.27529940500519834|
|(10,[0,1,2,3,4,5,...|  0.6183753593590278|
|(10,[0,1,2,3,4,5,...| -0.7249694165312423|
|(10,[0,1,2,3,4,5,...|  1.9846607199255222|
|(10,[0,1,2,3,4,5,...|  -2.224710856481673|
|(10,[0,1,2,3,4,5,...|   2.903066621032031|
|(10,[0,1,2,3,4,5,...|  1.5539534181436165|
|(10,[0,1,2,3,4,5,...|  1.7647900684963536|
|(10,[0,1,2,3,4,5,...|   0.999449826693239|
|(10,[0,1,2,3,4,5,...| -0.8549864358814567|
|(10,[0,1,2,3,4,5,...| -1.8043022378920774|
|(10,[0,1,2,3,4,5,...| -1.8754075818439322|
|(10,[0,1,2,3,4,5,...|-0.23582210622192312|
|(10,[0,1,2,3,4,5,...|  2.1608445906737175|
|(10,[0,1,2,3,4,5,...|  -3.299899318826038|
|(10,[0,1,2,3,4,5,...|  1.6388148161306586|
|(10,[0,1,2,3,4,5,...|   1.91452